In [35]:
import hashlib
import json
from time import time
from uuid import uuid4
# from config import CHAVE_PROFESSOR  # Importa a chave do arquivo config.py

In [36]:
CHAVE_PROFESSOR = "SENHA_SUPER_SECRETA"

In [41]:
class Transacao:
    def __init__(self, remetente, destinatario, valor, motivo, assinatura=None, timestamp=None):
        self.remetente = remetente
        self.destinatario = destinatario
        self.valor = valor
        self.motivo = motivo
        self.id = str(uuid4())
        self.timestamp = timestamp if timestamp is not None else time()  # Usa o fornecido ou gera um novo
        self.status = "PENDENTE"
        self.assinatura = assinatura

        if remetente == "PROFESSOR":
            if not self.validar_assinatura():
                raise Exception("Assinatura do professor inválida!")
            self.status = "APROVADA"

    def validar_assinatura(self):
      if self.remetente != "PROFESSOR":
         return False

      # Mensagem deve ser IDÊNTICA à usada em criar_transacao_professor()
      mensagem = f"PROFESSOR:{self.destinatario}:{self.valor}:{self.timestamp}"
      assinatura_correta = hashlib.sha256((mensagem + CHAVE_PROFESSOR).encode()).hexdigest()

      return self.assinatura == assinatura_correta

class Bloco:
    def __init__(self, index, transacoes, hash_anterior):
        self.index = index
        self.transacoes = transacoes
        self.hash_anterior = hash_anterior
        self.timestamp = time()
        self.nonce = 0
        self.hash_atual = self.calcular_hash()

    def calcular_hash(self):
        bloco_string = json.dumps({
            "index": self.index,
            "transacoes": [vars(t) for t in self.transacoes],
            "hash_anterior": self.hash_anterior,
            "timestamp": self.timestamp,
            "nonce": self.nonce
        }, sort_keys=True).encode()

        return hashlib.sha256(bloco_string).hexdigest()

class Blockchain:
    def __init__(self):
        self.cadeia = [self.criar_bloco_genesis()]
        self.transacoes_pendentes = []

    def criar_bloco_genesis(self):
        return Bloco(0, [], "0")

    def obter_ultimo_bloco(self):  # 👈 MÉTODO ADICIONADO
        return self.cadeia[-1]

    def adicionar_transacao(self, transacao):
        if transacao.remetente == "PROFESSOR" and transacao.status != "APROVADA":
            raise Exception("Transação do professor não autenticada!")
        self.transacoes_pendentes.append(transacao)
        return transacao.id

    def criar_bloco(self):
        transacoes_aprovadas = [
            t for t in self.transacoes_pendentes
            if t.status == "APROVADA"
        ]
        self.transacoes_pendentes = [
            t for t in self.transacoes_pendentes
            if t.status != "APROVADA"
        ]
        novo_bloco = Bloco(
            index=len(self.cadeia),
            transacoes=transacoes_aprovadas,
            hash_anterior=self.obter_ultimo_bloco().hash_atual  # 👈 Agora funciona!
        )
        self.cadeia.append(novo_bloco)
        return novo_bloco

    @staticmethod
    def criar_transacao_professor(destinatario, valor, motivo):
        timestamp = time()
        mensagem = f"PROFESSOR:{destinatario}:{valor}:{timestamp}"
        assinatura = hashlib.sha256((mensagem + CHAVE_PROFESSOR).encode()).hexdigest()
        return Transacao(
            remetente="PROFESSOR",
            destinatario=destinatario,
            valor=valor,
            motivo=motivo,
            assinatura=assinatura,
            timestamp=timestamp
        )
    def calcular_saldo(self, grupo):
        """Calcula o saldo baseado apenas nas transações aprovadas e incluídas nos blocos."""
        saldo = 0
        for bloco in self.cadeia:
            for transacao in bloco.transacoes:
                if transacao.destinatario == grupo:
                    saldo += transacao.valor
                if transacao.remetente == grupo:
                    saldo -= transacao.valor
        return saldo


In [42]:
# Inicializa a blockchain
blockchain = Blockchain()

# 1. Transação do professor (auto-aprovada)
t_prof = Blockchain.criar_transacao_professor(
    destinatario="Grupo1",
    valor=100,
    motivo="Presença"
)
blockchain.adicionar_transacao(t_prof)
bloco1 = blockchain.criar_bloco()
print(f"Bloco #{bloco1.index} criado com {len(bloco1.transacoes)} transações.")  # Deve ser 1

# 2. Transação de grupo (pendente)
t_grupo = Transacao(
    remetente="Grupo1",
    destinatario="Grupo2",
    valor=30,
    motivo="Troca de Servicos"
)
blockchain.adicionar_transacao(t_grupo)
print(f"Transação {t_grupo.id} criada. Status: {t_grupo.status}")  # PENDENTE

# 3. Cria bloco (não deve incluir pendentes)
bloco2 = blockchain.criar_bloco()
print(f"Bloco #{bloco2.index} criado com {len(bloco2.transacoes)} transações.")  # Deve ser 0

# 4. Aprovação MANUAL (simulação)
for i, t in enumerate(blockchain.transacoes_pendentes):
    if t.id == t_grupo.id:
        blockchain.transacoes_pendentes[i].status = "APROVADA"
        print(f"Transação {t.id} APROVADA manualmente!")  # Confirmação
        break

# 5. Cria bloco com a transação aprovada
bloco3 = blockchain.criar_bloco()
print(f"Bloco #{bloco3.index} criado com {len(bloco3.transacoes)} transações.")  # Deve ser 1

# 6. Verifica saldos
saldo_grupo1 = blockchain.calcular_saldo("Grupo1")
saldo_grupo2 = blockchain.calcular_saldo("Grupo2")
print(f"Saldo Grupo1: {saldo_grupo1} | Saldo Grupo2: {saldo_grupo2}")

# 7. Verifica transações no último bloco
print(f"Transações no bloco #{bloco3.index}:")
for t in bloco3.transacoes:
    print(f"- {t.remetente} → {t.destinatario}: {t.valor} ({t.motivo})")


Bloco #1 criado com 1 transações.
Transação b78b4bf2-6661-4206-bbd6-c23aa516fe2e criada. Status: PENDENTE
Bloco #2 criado com 0 transações.
Transação b78b4bf2-6661-4206-bbd6-c23aa516fe2e APROVADA manualmente!
Bloco #3 criado com 1 transações.
Saldo Grupo1: 70 | Saldo Grupo2: 30
Transações no bloco #3:
- Grupo1 → Grupo2: 30 (Troca de Servicos)
